In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
import chromadb
import openai
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI

# Set the API key manually in the environment
os.environ['OPENAI_API_KEY'] = 'your API key
api_key='your API key'

# Access the API key
client_oai = OpenAI(api_key=os.environ['OPENAI_API_KEY'])


In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [ ]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.7 MB/s eta 0:00:

In [ ]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
def check_openai_api_key(api_key):
    try:

        # Test the API by making a request to retrieve embeddings (a simple query)
        response = client_oai.embeddings.create(
            input="This is a test to check the API key",
            model="text-embedding-3-large"
        )

        print("API key is valid!")

    except client_oai.error.AuthenticationError:
        print("Invalid API key.")

    except Exception as e:
        print(f"An error occurred: {e}")

check_openai_api_key(api_key)

API key is valid!


In [ ]:
# Load the Excel file into a Pandas DataFrame
df = pd.read_excel('df_with_metadata.xlsx')

In [ ]:
# Preprocessing step to clean data by filling NaN values with 'No Data' and ensuring valid strings
df = df.fillna('No Data')  # Fill NaN values with 'No Data'
df = df.apply(lambda col: col.apply(lambda x: x if isinstance(x, str) and x.strip() else 'No Data'))
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
import os
import chromadb
from chromadb.config import Settings

In [ ]:
# Define a writable directory for ChromaDB storage
path = "Chroma_db_storage"  # Use a relative path

# Create the directory if it does not exist
os.makedirs(path, exist_ok=True)

# Create a persistent client with the specified directory for storage
client = chromadb.PersistentClient(path=path)

print(f"Persistent client created with storage at: {os.path.abspath(path)}")

Persistent client created with storage at: /content/Chroma_db_storage


In [ ]:
# Define the collection name
collection_name = "openai_sds_embeddings_metadata"

# Check if the collection already exists and delete it if it does
try:
    client.delete_collection(name=collection_name)
    print(f"Existing collection '{collection_name}' deleted.")
except Exception as e:
    print(f"No existing collection found or error deleting: {e}")

# Create a new collection
collection = client.create_collection(name=collection_name)

print(f"Collection '{collection_name}' created.")

No existing collection found or error deleting: Collection openai_sds_embeddings_metadata does not exist.
Collection 'openai_sds_embeddings_metadata' created.


In [ ]:

# Define the get_embedding function
def get_embedding(text, model="text-embedding-3-large"):
    client_oai = OpenAI()

    try:
        if isinstance(text, str) and text.strip():
            text = text.replace("\n", " ")
            tokens = len(text.split())
            if tokens > 8192:
                text = " ".join(text.split()[:8192])

            response = client_oai.embeddings.create(input=[text], model=model)
            return response.data[0].embedding
        return None
    except Exception as e:
        print(f"Embedding generation error: {e}")
        return None



In [ ]:
# Define section columns and corresponding query_property values
SECTION_COLUMNS = {
    "Identification": "identification",
    "Hazards identification": "hazards",
    "Composition/information on ingredients": "ingredients",
    "First aid measures": "first aid",
    "Firefighting measures": "firefighting",
    "Accidental release measures": "release",
    "Handling and storage": "storage",
    "Exposure controls/personal protection": "exposure",
    "Physical and chemical properties": "properties",
    "Stability and reactivity": "stability",
    "Toxicological information": "toxicology",
    "Ecological information": "ecology",
    "Disposal considerations": "disposal",
    "Transport information": "transport",
    "Regulatory information": "regulation",
    "Other information": "other"
}

def store_sds_embeddings_metadata(df):
    successful_stores = 0
    failed_stores = 0

    for index, row in df.iterrows():
        # Extract essential metadata
        product_name = str(row.get("Product Name", ""))
        manufacturer = str(row.get("Supplier Name", ""))
        file_name = str(row.get("File Name", ""))

        # Process each section and store only relevant metadata
        for section_name, query_property in SECTION_COLUMNS.items():
            section_content = row.get(section_name, "").strip()
            if not section_content:
                continue  # Skip empty sections

            # Generate embedding for the section content
            embedding = get_embedding(section_content)

            if embedding:
                section_index = list(SECTION_COLUMNS.keys()).index(section_name) + 1
                unique_id = f"{index}_{query_property}"

                # Prepare metadata based on API structure
                metadata_entry = {
                    "product_name": product_name,
                    "manufacturer": manufacturer,
                    "file_name": file_name,
                    "section_number": section_index,
                    "query_property": query_property,
                    "header": f"{section_name} Header",
                    "footer": f"{section_name} Footer"
                }

                # Store embedding with document content directly
                collection.add(
                    embeddings=[embedding],
                    documents=[section_content],  # Directly store section content as document
                    ids=[unique_id],
                    metadatas=[metadata_entry]
                )
                successful_stores += 1
                print(f"Stored embedding for {product_name} by {manufacturer}, section {query_property}")
            else:
                failed_stores += 1

    print(f"\nEmbedding storage completed with {successful_stores} successes and {failed_stores} failures")

In [ ]:
# Print initial collection size
print(f"Initial collection size: {collection.count()}")


store_sds_embeddings_metadata(df)

# Print final collection size
print(f"Final collection size: {collection.count()}")

Initial collection size: 0
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section identification
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section hazards
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section ingredients
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section first aid
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section firefighting
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section release
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section storage
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section exposure
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section properties
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section stability
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section toxicology
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section ecology
Stored embedding for 4-Aminopyridine by /UNDERTAKING., section disposal
Stored embedding for 4

In [ ]:
# List all collections in the persistent client
collections = client.list_collections()
print(f"Existing collections: {[col.name for col in collections]}")

Existing collections: ['openai_sds_embeddings_metadata']


In [ ]:
# Function to view collection items
def view_collection_items(collection, num_items=10):
    """
    View a sample of items from the ChromaDB collection.

    Parameters:
    - collection: The ChromaDB collection object.
    - num_items: Number of items to retrieve for display (default is 10).
    """
    try:
        # Get the items from the collection (limited by num_items)
        results = collection.get(
            ids=None,  # Retrieve all items (can be limited)
            limit=num_items,  # Limit to num_items for display
            include=['embeddings', 'metadatas', 'documents']  # What to include in the results
        )

        # Print the retrieved items
        for idx, metadata in enumerate(results['metadatas']):
            print(f"\nItem {idx + 1}:")
            print(f"ID: {results['ids'][idx]}")
            print(f"Metadata: {metadata}")
            print(f"Document: {results['documents'][idx]}")
            print(f"Embedding Length: {len(results['embeddings'][idx])} (First 5 values: {results['embeddings'][idx][:5]})")

        print(f"\nTotal items retrieved: {len(results['ids'])}")

    except Exception as e:
        print(f"Error retrieving collection items: {e}")

In [ ]:
# Function to filter collection items by content
def filter_collection_by_document_content(collection, document_filter, num_items=10):
    """
    Filter and view items from the ChromaDB collection based on document content.

    Parameters:
    - collection: The ChromaDB collection object.
    - document_filter: Dictionary to filter by document content.
    - num_items: Number of items to retrieve for display (default is 10).
    """
    try:
        # Get the filtered items from the collection (limited by num_items)
        results = collection.get(
            ids=None,  # Retrieve all matching items
            limit=num_items,  # Limit to num_items for display
            where_document=document_filter,  # Filter based on document content
            include=['embeddings', 'metadatas', 'documents']  # What to include in the results
        )

        # Print the retrieved items
        for idx, metadata in enumerate(results['metadatas']):
            print(f"\nItem {idx + 1}:")
            print(f"ID: {results['ids'][idx]}")
            print(f"Metadata: {metadata}")
            print(f"Document: {results['documents'][idx]}")
            print(f"Embedding Length: {len(results['embeddings'][idx])} (First 5 values: {results['embeddings'][idx][:5]})")

        print(f"\nTotal items retrieved: {len(results['ids'])}")

    except Exception as e:
        print(f"Error retrieving collection items: {e}")

In [ ]:
# Function to filter collection items by metadata and content
def filter_collection_by_metadata_and_content(collection, metadata_filter, document_content_filters, num_items=10):
    """
    Filter and view items from the ChromaDB collection based on metadata and document content.

    Parameters:
    - collection: The ChromaDB collection object.
    - metadata_filter: Dictionary to filter by metadata.
    - document_content_filters: Dictionary to filter by multiple conditions in document content.
    - num_items: Number of items to retrieve for display (default is 10).
    """
    try:
        # Get the filtered items from the collection (limited by num_items)
        results = collection.get(
            ids=None,  # Retrieve all matching items
            limit=num_items,  # Limit to num_items for display
            where=metadata_filter,  # Filter based on metadata
            where_document=document_content_filters,  # Filter based on document content
            include=['embeddings', 'metadatas', 'documents']  # What to include in the results
        )

        # Print the retrieved items
        for idx, metadata in enumerate(results['metadatas']):
            print(f"\nItem {idx + 1}:")
            print(f"ID: {results['ids'][idx]}")
            print(f"Metadata: {metadata}")
            print(f"Document: {results['documents'][idx]}")
            print(f"Embedding Length: {len(results['embeddings'][idx])} (First 5 values: {results['embeddings'][idx][:5]})")

        print(f"\nTotal items retrieved: {len(results['ids'])}")

    except Exception as e:
        print(f"Error retrieving collection items: {e}")

In [ ]:
collection_name = "openai_sds_embeddings_metadata"
collection = client.get_collection(name=collection_name)

# Usage: Return a given number of items
view_collection_items(collection, num_items=18)


Item 1:
ID: 0_identification
Metadata: {'file_name': '0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2', 'footer': 'Identification Footer', 'header': 'Identification Header', 'manufacturer': '/UNDERTAKING.', 'product_name': '4-Aminopyridine', 'query_property': 'identification', 'section_number': 1}
Document: IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF THE COMPANY/UNDERTAKING.
1.1. Product identifier
PRODUCT NAME
: 4-Aminopyridine
CAS RN
: 504-24-5
EC#
: 207-987-9
SYNONYMS
: y-pyridylamine; Gamma-aminopyridine; m-aminopyridine; amino-4-pyridine
SYSTEMATIC NAME MOLECULAR FORMULA STRUCTURAL FORMULA
: 4-Aminopyridine, 4-Pyridinamine, 4-Pyridylamine, Pyridine : C5H6N2 :
NH2
N
1.2. Relevant identified uses of the substance or mixture and uses advised against
1.2.1. Relevant identified uses 4-Aminopyridine is used as an Intermediate, Bird Repellant avicide and in Medication.
Uses advised against: None
1.3. Details of the supplier of the safety data sheet Jubilant Ingrevia Limited FACT

In [ ]:
# Example usage:
metadata_filter = {
    "product_name": "(+)-trans-1,2-Diaminocyclohexane"  # Filter by specific product name
}

document_filter = {
    "$and": [  # Use the $and operator to combine multiple conditions
        {"$contains": " eye"},  # Condition for "Skin Corrosion"
        {"$contains": "ph"}  # Condition for "Serious Eye Damage"
    ]
}

filter_collection_by_metadata_and_content(collection, metadata_filter, document_filter)


Item 1:
ID: 1_hazards
Metadata: {'file_name': '1-2-cyclohexanediamine-sds-fisher_2', 'footer': 'Hazards identification Footer', 'header': 'Hazards identification Header', 'manufacturer': 'of the safety data sheet\nCompany Fisher Scientific Company One Reagent Lane Fair Lawn, NJ 07410 Tel: (201) 796-7100 Emergency Telephone Number For information US call: 001-800-ACROS-01 / Europe call: +32 14 57 52 11 Emergency Number US:001-201-796-7100 / Europe: +32 14 57 52 99 CHEMTREC Tel. No.US:001-800-424-9300 / Europe:001-703-527-3887 Acros Organics One Reagent Lane Fair Lawn, NJ 07410', 'product_name': '(+)-trans-1,2-Diaminocyclohexane', 'query_property': 'hazards', 'section_number': 2}
Document: 2. Hazard(s) identification
Classification
This chemical is considered hazardous by the 2012 OSHA Hazard Communication Standard (29 CFR 1910.1200)
Flammable liquids
Skin Corrosion/Irritation
Serious Eye Damage/Eye Irritation
Label Elements
Signal Word Danger Hazard Statements Combustible liquid Causes

In [ ]:
# Get collection info
print("Collection name:", collection.name)
print("Collection count:", collection.count())

# Get sample entries to understand structure
print("\nExample entries:")
sample_results = collection.get(limit=2)
print("\nStructure of results:")
for key, value in sample_results.items():
    print(f"\n{key}:")
    print(value)

Collection name: openai_sds_embeddings_metadata
Collection count: 800

Example entries:

Structure of results:

ids:
['0_identification', '0_hazards']

embeddings:
None

documents:
['IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF THE COMPANY/UNDERTAKING.\n1.1. Product identifier\nPRODUCT NAME\n: 4-Aminopyridine\nCAS RN\n: 504-24-5\nEC#\n: 207-987-9\nSYNONYMS\n: y-pyridylamine; Gamma-aminopyridine; m-aminopyridine; amino-4-pyridine\nSYSTEMATIC NAME MOLECULAR FORMULA STRUCTURAL FORMULA\n: 4-Aminopyridine, 4-Pyridinamine, 4-Pyridylamine, Pyridine : C5H6N2 :\nNH2\nN\n1.2. Relevant identified uses of the substance or mixture and uses advised against\n1.2.1. Relevant identified uses 4-Aminopyridine is used as an Intermediate, Bird Repellant avicide and in Medication.\nUses advised against: None\n1.3. Details of the supplier of the safety data sheet Jubilant Ingrevia Limited FACTORY & REGISTERED OFFICE: Jubilant Ingrevia Limited., Bhartiagram, Gajraula, District: Amroha, Uttar Pradesh-244223,

In [ ]:
results

{'status': 'error',
 'message': "Expected where to have exactly one operator, got {'product_name': '(+)-trans-1,2-Diaminocyclohexane', 'query_property': 'identification'} in get.",
 'results': []}

In [ ]:
def retrieve_sds_info(collection, product_name=None, manufacturer=None, section_number=None, query_property=None, n_results=5):
    """
    Retrieve SDS information based on metadata filters.

    Args:
        collection: ChromaDB collection object
        product_name (str, optional): Name of the product
        manufacturer (str, optional): Name of the manufacturer
        section_number (int, optional): Section number (1-16)
        query_property (str, optional): Query property (hazards, first_aid, etc.)
        n_results (int, optional): Number of results to return. Defaults to 5.
    """
    try:
        # Build where clause using ChromaDB's where operators
        where_conditions = {}

        if product_name:
            where_conditions = {"product_name": {"$eq": product_name}}
        if manufacturer:
            where_conditions = {"manufacturer": {"$eq": manufacturer}}
        if section_number:
            where_conditions = {"section_number": {"$eq": section_number}}
        if query_property:
            where_conditions = {"query_property": {"$eq": query_property}}

        # Get results from collection
        results = collection.get(
            where=where_conditions if where_conditions else None,
            limit=n_results,
            include=['documents', 'metadatas']
        )

        # Format results for easier consumption
        formatted_results = []
        for i in range(len(results['ids'])):
            formatted_results.append({
                'id': results['ids'][i],
                'document': results['documents'][i],
                'metadata': results['metadatas'][i]
            })

        return {
            'status': 'success',
            'count': len(formatted_results),
            'results': formatted_results
        }

    except Exception as e:
        return {
            'status': 'error',
            'message': str(e),
            'results': []
        }

# Example usage:
def print_sds_results(results):
    """Helper function to print results in a readable format"""
    if results['status'] == 'success':
        print(f"Found {results['count']} results:")
        for i, result in enumerate(results['results'], 1):
            print(f"\nResult {i}:")
            print(f"Product: {result['metadata']['product_name']}")
            print(f"Section: {result['metadata']['query_property']}")
            print(f"File: {result['metadata']['file_name']}")
            print("Content Preview:", result['document'][:200], "...")
    else:
        print(f"Error: {results['message']}")

# Example queries:

# Get identification information for a specific produ

In [ ]:
resultss = retrieve_sds_info(
    collection,
    product_name="CESL-30N",
    query_property="hazards",
    manufacturer = "/Importer/Supplier/Distributor information Address: 4-4-9 Kitahama, Chuo-ku, Osaka Country: Japan Telephone Number: +81-6-6206-3311"
)


print_sds_results(resultss)


Found 5 results:

Result 1:
Product: 4-Aminopyridine
Section: hazards
File: 0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2
Content Preview: SECTION 2: HAZARD(S) IDENTIFICATION.
2.1. Classification of the substance or mixture
GHS-US classification
Acute Toxicity Oral (Category 2)
H300 -
Acute Toxicity Dermal (Category 3)
H311-
Acute Toxici ...

Result 2:
Product: (+)-trans-1,2-Diaminocyclohexane
Section: hazards
File: 1-2-cyclohexanediamine-sds-fisher_2
Content Preview: 2. Hazard(s) identification
Classification
This chemical is considered hazardous by the 2012 OSHA Hazard Communication Standard (29 CFR 1910.1200)
Flammable liquids
Skin Corrosion/Irritation
Serious E ...

Result 3:
Product: 2-Aminopyrimidine
Section: hazards
File: 2-aminopyrimidine-sds_2
Content Preview: 2: Hazards identification
2.1 Classification of the substance or mixture Not a hazardous substance or mixture.
2.2 GHS Label elements, including precautionary statements Not a hazardous substance or m ...

Result 

#**WORKING FROM HERE: **

In [ ]:
import pandas as pd
import chromadb
import ast
import os
from chromadb.utils import embedding_functions
from typing import Dict, List

# Section ID to query_property mapping
SECTION_COLUMNS = {
    1: "identification",
    2: "hazards",
    3: "ingredients",
    4: "first aid",
    5: "firefighting",
    6: "release",
    7: "storage",
    8: "exposure",
    9: "properties",
    10: "stability",
    11: "toxicology",
    12: "ecology",
    13: "disposal",
    14: "transport",
    15: "regulation",
    16: "other"
}

class SDSProcessor:
    def __init__(self, chroma_db_path: str = "chroma_db"):
        # Initialize ChromaDB with OpenAI embeddings
        openai_ef = embedding_functions.OpenAIEmbeddingFunction(
            api_key=os.environ["OPENAI_API_KEY"],
            model_name="text-embedding-3-large"
        )

        # Initialize ChromaDB client
        self.client = chromadb.PersistentClient(path=chroma_db_path)

        # Create or get collection
        try:
            self.collection = self.client.get_collection(
                name="sds_documents",
                embedding_function=openai_ef
            )
        except:
            self.collection = self.client.create_collection(
                name="sds_documents",
                embedding_function=openai_ef
            )

    def process_document_column(self, df: pd.DataFrame) -> List[Dict]:
        """Process the Document column from DataFrame"""
        processed_documents = []

        for _, row in df.iterrows():
            try:
                # Convert string representation of list to actual list
                doc_list = ast.literal_eval(row['Document'])

                # Process each dictionary in the list
                for doc in doc_list:
                    # Add query_property based on section_id
                    metadata = doc['metadata'].copy()
                    metadata['query_property'] = SECTION_COLUMNS[metadata['section_id']]

                    processed_documents.append({
                        'page_content': doc['page_content'],
                        'metadata': metadata
                    })

            except Exception as e:
                print(f"Error processing row: {str(e)}")
                continue

        return processed_documents

    def store_documents(self, documents: List[Dict]):
        """Store processed documents in ChromaDB"""
        if not documents:
            print("No documents to store!")
            return

        texts = []
        metadatas = []
        ids = []

        for doc in documents:
            texts.append(doc['page_content'])
            metadatas.append(doc['metadata'])
            ids.append(doc['metadata']['id'])

        # Store in batches
        batch_size = 100
        for i in range(0, len(texts), batch_size):
            batch_end = min(i + batch_size, len(texts))
            self.collection.add(
                documents=texts[i:batch_end],
                metadatas=metadatas[i:batch_end],
                ids=ids[i:batch_end]
            )
            print(f"Stored batch {i//batch_size + 1}")

    def query_documents(self,
                       product_name: str = None,
                       query_property: str = None,
                       file_name: str = None) -> List[Dict]:
        """Query documents"""
        # Use single where clause based on priority
        where_clause = None
        if product_name:
            where_clause = {"product_name": {"$eq": product_name}}
        elif query_property:
            where_clause = {"query_property": {"$eq": query_property}}
        elif file_name:
            where_clause = {"file_name": {"$eq": file_name}}

        results = self.collection.get(
            where=where_clause,
            include=['documents', 'metadatas']
        )

        # Additional filtering in memory if needed
        if product_name and query_property:
            results = self._filter_results(
                results,
                lambda x: (x['product_name'] == product_name and
                          x['query_property'] == query_property)
            )

        return self._format_results(results)

    def _filter_results(self, results: Dict, filter_fn) -> Dict:
        """Filter results in memory"""
        filtered = {
            'ids': [],
            'documents': [],
            'metadatas': []
        }

        for i, metadata in enumerate(results['metadatas']):
            if filter_fn(metadata):
                filtered['ids'].append(results['ids'][i])
                filtered['documents'].append(results['documents'][i])
                filtered['metadatas'].append(metadata)

        return filtered

    def _format_results(self, results: Dict) -> List[Dict]:
        """Format results"""
        return [{
            'id': results['ids'][i],
            'content': results['documents'][i],
            'metadata': results['metadatas'][i]
        } for i in range(len(results['ids']))]

def main():
    # Read Excel file
    df = pd.read_excel('df_with_metadata.xlsx')

    # Initialize processor
    processor = SDSProcessor()

    # Process documents
    documents = processor.process_document_column(df)
    print(f"Processed {len(documents)} documents")

    if documents:
        # Store in ChromaDB
        processor.store_documents(documents)

        # Example queries
        print("\nQuerying hazards information for 4-Aminopyridine:")
        results = processor.query_documents(
            product_name="4-Aminopyridine",
            query_property="hazards"
        )

        for result in results:
            print(f"\nProduct: {result['metadata']['product_name']}")
            print(f"Section: {result['metadata']['query_property']}")
            print(f"Content preview: {result['content'][:200]}...")

    return documents

if __name__ == "__main__":
    main()

Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: malformed node or string on line 1: <ast.Name object at 0x7cd360fd7640>
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line

In [ ]:
# Test with small sample first
df_test = df.head(1)  # Just process first row
processor = SDSProcessor()
documents = processor.process_document_column(df_test)
print(f"Processed {len(documents)} documents from first row")

# Print first document structure
if documents:
    print("\nFirst document metadata:")
    print(documents[0]['metadata'])
    print("\nContent preview:")
    print(documents[0]['page_content'][:200])

Processed 16 documents from first row

First document metadata:
{'id': '1_1', 'section_id': 1, 'file_name': '0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2', 'supplier': '/UNDERTAKING.', 'product_name': '4-Aminopyridine', 'query_property': 'identification'}

Content preview:
IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF THE COMPANY/UNDERTAKING.
1.1. Product identifier
PRODUCT NAME
: 4-Aminopyridine
CAS RN
: 504-24-5
EC#
: 207-987-9
SYNONYMS
: y-pyridylamine; Gamma-aminop


In [ ]:
def retrieve_sds_content(collection, file_name=None, product_name=None, supplier=None, query_property=None):
    """
    Retrieve SDS content with various filters.

    Args:
        collection: ChromaDB collection
        file_name: Name of the file
        product_name: Name of the product
        supplier: Name of manufacturer/supplier
        query_property: Section type (e.g., "hazards", "first aid", etc.)

    Returns:
        List of dictionaries containing matching content
    """
    # Build where clause based on single primary filter
    where_clause = None
    if file_name:
        where_clause = {"file_name": {"$eq": file_name}}
    elif product_name:
        where_clause = {"product_name": {"$eq": product_name}}
    elif supplier:
        where_clause = {"supplier": {"$eq": supplier}}
    elif query_property:
        where_clause = {"query_property": {"$eq": query_property}}

    # Get initial results
    results = collection.get(
        where=where_clause,
        include=['documents', 'metadatas']
    )

    # Apply additional filters in memory if needed
    filtered_results = []
    for i in range(len(results['ids'])):
        metadata = results['metadatas'][i]
        content = results['documents'][i]

        # Check if result matches all provided filters
        matches = True
        if file_name and metadata['file_name'] != file_name:
            matches = False
        if product_name and metadata['product_name'] != product_name:
            matches = False
        if supplier and metadata['supplier'] != supplier:
            matches = False
        if query_property and metadata['query_property'] != query_property:
            matches = False

        if matches:
            filtered_results.append({
                'content': content,
                'metadata': metadata
            })

    return filtered_results

# Helper function to print results in a readable format
def print_sds_results(results):
    """Print SDS results in a readable format"""
    if not results:
        print("No matching results found.")
        return

    print(f"\nFound {len(results)} matching sections:")
    for i, result in enumerate(results, 1):
        print(f"\n--- Result {i} ---")
        print(f"Product: {result['metadata']['product_name']}")
        print(f"Section: {result['metadata']['query_property']}")
        print(f"File: {result['metadata']['file_name']}")
        print(f"Supplier: {result['metadata']['supplier']}")
        print("\nContent:")
        print(result['content'])
        print("-" * 80)

# Example usage
def main():
    # Initialize ChromaDB collection (assuming it's already created and populated)
    client = chromadb.PersistentClient(path="chroma_db")
    collection = client.get_collection("sds_documents")

    # Example 1: Get hazards information for a specific product
    print("\nExample 1: Hazards information for 4-Aminopyridine")
    results = retrieve_sds_content(
        collection,
        product_name="4-Aminopyridine",
        query_property="hazards"
    )
    print_sds_results(results)

    # Example 2: Get all sections for a specific file
    print("\nExample 2: All sections from specific file")
    results = retrieve_sds_content(
        collection,
        file_name="0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2"
    )
    print_sds_results(results)

    # Example 3: Get all documents from a specific supplier
    print("\nExample 3: All documents from specific supplier")
    results = retrieve_sds_content(
        collection,
        supplier="/UNDERTAKING."
    )
    print_sds_results(results)

    # Example 4: Get specific section from specific manufacturer
    print("\nExample 4: First aid measures from specific manufacturer")
    results = retrieve_sds_content(
        collection,
        supplier="/UNDERTAKING.",
        query_property="first aid"
    )
    print_sds_results(results)

if __name__ == "__main__":
    main()


Example 1: Hazards information for 4-Aminopyridine

Found 1 matching sections:

--- Result 1 ---
Product: 4-Aminopyridine
Section: hazards
File: 0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2
Supplier: /UNDERTAKING.

Content:
SECTION 2: HAZARD(S) IDENTIFICATION.
2.1. Classification of the substance or mixture
GHS-US classification
Acute Toxicity Oral (Category 2)
H300 -
Acute Toxicity Dermal (Category 3)
H311-
Acute Toxicity Inhalation (Category 3)
H331-
Skin irritation (Category 1)
H314 -
Eye irritation (Category 1)
H318 -
Specific target organ toxicity
H335 -
- single exposure (Category 3)
Hazardous to the aquatic environment (long-term)
(Category 2)
2.2. Label elements
Hazard Pictograms: GHS 06, GHS 05, GHS09
H411-
Signal word: Danger!
Jubilant Ingrevia Limited
Fatal if swallowed. Toxic in contact with skin Toxic if inhaled Causes severe skin burns and eye damage. Causes serious eye damage. May cause respiratory irritation.
Toxic to aquatic life with long lasting effects.
Pag

In [ ]:
# Initialize ChromaDB
client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_collection("sds_documents")

# Example queries:

# 1. Get hazards information for a specific product
results = retrieve_sds_content(
    collection,
    product_name="3,4-Diaminopyridine",
    supplier = 'TCI America',
    query_property="identification"
)

results

[{'content': 'IDENTIFICATION\nProduct name:\n3,4-Diaminopyridine\nProduct code:\nD1149\nProduct use:\nRestrictions on use:\nCompany:\nTCI America\n9211 N. Harborgate Street\nPortland, OR 97203 U.S.A.\nTelephone:\n+1-800-423-8616 / +1-503-283-1681\nFax:\n+1-888-520-1075 / +1-503-283-1987\ne-mail:\nsales-US@TCIchemicals.com\nwww.TCIchemicals.com',
  'metadata': {'file_name': '3-4-diaminopyridine-tci-sds-d1149_us_en_2',
   'id': '4_1',
   'product_name': '3,4-Diaminopyridine',
   'query_property': 'identification',
   'section_id': 1,
   'supplier': 'TCI America'}}]

In [ ]:
import pandas as pd
import json
import os
from typing import Dict, List
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Section ID to query_property mapping
SECTION_COLUMNS = {
    1: "identification",
    2: "hazards",
    3: "ingredients",
    4: "first aid",
    5: "firefighting",
    6: "release",
    7: "storage",
    8: "exposure",
    9: "properties",
    10: "stability",
    11: "toxicology",
    12: "ecology",
    13: "disposal",
    14: "transport",
    15: "regulation",
    16: "other"
}

class SDSProcessor:
    def __init__(self, persist_directory: str = "chroma_db"):
        self.embeddings = OpenAIEmbeddings()
        self.persist_directory = persist_directory
        self.db = None

    def process_document_column(self, df: pd.DataFrame) -> List[Dict]:
        """Process the Document column from DataFrame"""
        processed_documents = []

        for _, row in df.iterrows():
            try:
                # Try to parse as JSON first
                try:
                    doc_list = json.loads(row['Document'])
                except:
                    # If JSON fails, try literal_eval but with string cleaning
                    import ast
                    doc_str = str(row['Document']).strip()
                    doc_list = ast.literal_eval(doc_str)

                # Process each dictionary in the list
                for doc in doc_list:
                    metadata = doc['metadata'].copy()
                    metadata['query_property'] = SECTION_COLUMNS[metadata['section_id']]

                    processed_documents.append({
                        'page_content': doc['page_content'],
                        'metadata': metadata
                    })

            except Exception as e:
                print(f"Error processing row: {str(e)}")
                continue

        return processed_documents

    def store_documents(self, documents: List[Dict]):
        """Store processed documents in ChromaDB"""
        if not documents:
            print("No documents to store!")
            return

        texts = [doc['page_content'] for doc in documents]
        metadatas = [doc['metadata'] for doc in documents]

        self.db = Chroma.from_texts(
            texts=texts,
            embedding=self.embeddings,
            metadatas=metadatas,
            persist_directory=self.persist_directory
        )
        print(f"Stored {len(texts)} documents")

def retrieve_sds_content(db, file_name=None, product_name=None, supplier=None, query_property=None):
    """
    Retrieve SDS content using LangChain's Chroma without duplicates

    Args:
        db: LangChain's Chroma instance
        file_name: Name of the file
        product_name: Name of the product
        supplier: Name of manufacturer/supplier
        query_property: Section type (e.g., "hazards", "first aid", etc.)
    """
    # Choose the most selective filter as primary
    where_clause = {}

    if query_property and product_name:
        where_clause = {"query_property": query_property}
    elif product_name:
        where_clause = {"product_name": product_name}
    elif file_name:
        where_clause = {"file_name": file_name}
    elif supplier:
        where_clause = {"supplier": supplier}
    elif query_property:
        where_clause = {"query_property": query_property}

    # Get results using the primary filter
    if where_clause:
        primary_results = db.get(where=where_clause)
    else:
        primary_results = db.get()

    # Create a set to track unique metadata IDs
    seen_metadata_ids = set()

    # Apply additional filters and remove duplicates
    results = []
    for i, doc in enumerate(primary_results['documents']):
        metadata = primary_results['metadatas'][i]
        metadata_id = metadata['id']  # Using the metadata ID instead of Chroma's generated ID

        # Skip if we've seen this metadata ID
        if metadata_id in seen_metadata_ids:
            continue

        # Check all conditions
        matches = True
        if file_name and metadata['file_name'] != file_name:
            matches = False
        if product_name and metadata['product_name'] != product_name:
            matches = False
        if supplier and metadata['supplier'] != supplier:
            matches = False
        if query_property and metadata['query_property'] != query_property:
            matches = False

        if matches:
            seen_metadata_ids.add(metadata_id)
            results.append({
                'id': metadata_id,
                'content': doc,
                'metadata': metadata
            })

    return results

def print_sds_results(results):
    """Print SDS results in a readable format"""
    if not results:
        print("No matching results found.")
        return

    print(f"\nFound {len(results)} unique matching sections:")
    for i, result in enumerate(results, 1):
        print(f"\n--- Result {i} ---")
        print(f"Metadata ID: {result['id']}")
        print(f"Product: {result['metadata']['product_name']}")
        print(f"Section: {result['metadata']['query_property']}")
        print(f"File: {result['metadata']['file_name']}")
        print(f"Supplier: {result['metadata']['supplier']}")
        print("\nContent:")
        print(result['content'])
        print("-" * 80)

In [ ]:
    os.environ["OPENAI_API_KEY"] = "sk-svcacct-QsCZJG1m7apYpNXmYDj73kS-eon0tOJ9Iy6XuNwWD8dH3IDn9QM_OgxlYV-O2-PlAET3BlbkFJ5LFfldkzioy0xrvORw_jOaQo2acpnCix3KApdmur4QmECN1t17iJA0nBrv6cSS74MA"

    # Store data
    df = pd.read_excel('df_with_metadata.xlsx')
    processor = SDSProcessor()
    documents = processor.process_document_column(df)
    processor.store_documents(documents)


Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: malformed node or string on line 1: <ast.Name object at 0x7f024bbb2b00>
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error processing row: unterminated string literal (detected at line 1) (<unknown>, line

In [ ]:
    # Example retrievals
    db = processor.db
    # Example 1: Get hazards section for specific product
    results = retrieve_sds_content(db, product_name="Aniline", query_property="hazards", supplier='TCI America', )
    print_sds_results(results)



Found 1 unique matching sections:

--- Result 1 ---
Metadata ID: 15_2
Product: Aniline
Section: hazards
File: aniline-tci-a0463_us_en-sds_2
Supplier: TCI America

Content:
2. HAZARD(S) IDENTIFICATION
OSHA Haz Com: CFR 1910.1200: WHMIS 2015: Acute Toxicity - Oral [Category 3] Acute Toxicity - Dermal [Category 3] Acute Toxicity - Inhalation [Category 2] Eye Damage/Irritation [Category 1] Sensitization - Skin [Category 1] Germ Cell Mutagenicity [Category 2] Carcinogenicity [Category 2] Specific Target Organ Toxicity (Single Exposure) [Category 1] Specific Target Organ Toxicity (Repeated Exposure) [Category 1] Flammable Liquids [Category 4] Aquatic Hazard (Acute) [Category 1]
Signal word: Danger!
Hazard Statement(s): Combustible liquid Fatal if inhaled Toxic if swallowed or in contact with skin Causes serious eye damage May cause an allergic skin reaction Suspected of causing genetic defects Suspected of causing cancer Very toxic to aquatic life Causes damage to: Liver Blood System Respir

In [ ]:
example = retrieve_sds_content(db, product_name="3,4-Diaminopyridine", query_property="hazards", supplier='TCI America', )


In [ ]:
example

[{'id': '4_2',
  'content': '2. HAZARD(S) IDENTIFICATION\nOSHA Haz Com: CFR 1910.1200:\nWHMIS 2015:\nAcute Toxicity - Oral [Category 2]\nAcute Toxicity - Inhalation [Category 2]\nSkin Corrosion/Irritation [Category 2]\nEye Damage/Irritation [Category 2A]\nDanger!\nSignal word:\nHazard Statement(s):\nPictogram(s) or Symbol(s):\n!\nPrecautionary Statement(s): [Prevention]\n[Response]\n[Storage] [Disposal]\nTCI AMERICA SAFETY DATA SHEET\nFor laboratory research purposes.\nNot for drug or household use.\nFatal if swallowed or if inhaled\nCauses skin irritation\nCauses serious eye irritation\nPage 1 of 5\nEmergency telephone number: Chemical Emergencies: TCI America (8:00am - 5:00pm) PST +1-503-286-7624 Transportation Emergencies: Chemtrec 24-Hour +1-800-424-9300 (U.S.A.) +1-703-527-3887 (International) Responsible department: TCI America Environmental Health Safety and Security +1- 503-286-7624\nDo not breathe dust, fume, mist, vapors or spray. Use only outdoors or in a well-ventilated are